**Note: Try to avoid *GROUP BY* clause to solve the problems**

For the problems use the *Health Insurance Claim* dataset. You can get the details as well as the dataset from [here](https://www.kaggle.com/datasets/thedevastator/insurance-claim-analysis-demographic-and-health).

### **Problem 1:** What are the top 5 patients who claimed the highest insurance amounts?

### **Problem 2:** What is the average insurance claimed by patients based on the number of children they have?

### **Problem 3:** What is the highest and lowest claimed amount by patients in each region?

### **Problem 4:** What is the percentage of smokers in each age group?

### **Problem 5:** What is the difference between the claimed amount of each patient and the first claimed amount of that patient?

### **Problem 6:** For each patient, calculate the difference between their claimed amount and the average claimed amount of patients with the same number of children.

### **Problem 7:** Show the patient with the highest BMI in each region and their respective rank.

### **Problem 8:** Calculate the difference between the claimed amount of each patient and the claimed amount of the patient who has the highest BMI in their region.

### **Problem 9:** For each patient, calculate the difference in claim amount between the patient and the patient with the highest claim amount among patients with the same bmi and smoker status, within the same region. Return the result in descending order difference.

### **Problem 10:** For each patient, find the maximum BMI value among their next three records (ordered by age).

### **Problem 11:** For each patient, find the rolling average of the last 2 claims.

### **Problem 12:** Find the first claimed insurance value for male and female patients, within each region order the data by patient age in ascending order, and only include patients who are non-diabetic and have a bmi value between 25 and 30.

In [ ]:
select *, rank() over(order by claim) as rank1 from
insurance
limit 5

In [ ]:
select children,avg_claim from (select children, AVG(claim) over(partition by children ) as avg_claim ,row_number() over(partition by children ) as row_no  from
insurance)as t
where t.row_no =1

In [ ]:
select region,claim,minim,maxm from (select *, first_value(claim) over(partition by region order by claim desc) as maxm ,
last_value(claim) over(partition by region ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING  ) as minim,
row_number() over(partition by region) as row_no
from  insurance)as t
where t.row_no =1

In [ ]:
select t1.age,round((number1/(number+ number1))*100,2) as smoker_percentage from (select smoker,age,number
from (select *,count(age) over (partition by smoker,age order by age) as number,
row_number() over (partition by smoker,age order by age) as row_n from insurance) as t
where row_n =1 and smoker= "No"
group by smoker,age) as t1
join (select smoker,age,number1
from (select *,count(age) over (partition by smoker,age order by age) as number1,
row_number() over (partition by smoker,age order by age) as row_n from insurance) as t
where row_n =1 and smoker= "Yes"
group by smoker,age) as t2
on t1.age=t2.age

In [ ]:
select *,
claim - first_value(claim) over() as diffrence
from  insurance

In [ ]:
select *,
claim-avg(claim) over(partition by children)  as diffrence
from  insurance

In [ ]:
select * from (select *,rank() over(partition by region order by bmi desc) as rnk
	 from  insurance) as t
     where t.rnk = 1

In [ ]:
select *,
claim-first_value(claim) over(partition by region order by bmi desc ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) as"diff_claim"
from  insurance

In [ ]:
select *,
first_value(claim) over(partition by region,smoker order by  claim  desc)-claim as"diff"
from  insurance
order by diff desc

In [ ]:
SELECT *,
max(bmi) OVER(ORDER BY age ROWS BETWEEN 1 following AND 3 FOLLOWING)
FROM insurance

In [ ]:
SELECT *,
avg(claim) OVER(ROWS BETWEEN 2 PRECEDING AND 1 PRECEDING)
FROM insurance

In [ ]:
select region,gender,1st from (SELECT *,
first_value(claim) OVER(partition by gender,region order by age asc) as "1st",
row_number() OVER(partition by gender,region order by age asc) as rw
FROM insurance
where (bmi between 25 and 30) and diabetic="No") as t
where t.rw=1
order by region;